In [1]:
!pip install mwparserfromhell

In [5]:
import os

from datasets import load_dataset, DownloadConfig
import numpy as np
from transformers import AutoTokenizer
from scripts.chunk import chunk_and_tokenize

In [5]:
print(os.environ)

environ({'USER': 'lucia', 'SSH_CLIENT': '119.18.1.138 12430 22', 'SHLVL': '1', 'MOTD_SHOWN': 'pam', 'HOME': '/home/lucia', 'OLDPWD': '/home/lucia', 'SSL_CERT_FILE': '/usr/lib/ssl/certs/ca-certificates.crt', 'LOGNAME': 'lucia', '_': '/mnt/ssd-1/lucia/miniconda3/envs/3.10/bin/python', 'VSCODE_CLI_REQUIRE_TOKEN': '2f723f8f-d707-4f55-b012-db8b709ae4b4', 'PATH': '/mnt/ssd-1/lucia/miniconda3/envs/3.10/bin:/home/lucia/.vscode-server/cli/servers/Stable-863d2581ecda6849923a2118d93a088b0745d9d6/server/bin/remote-cli:/mnt/ssd-1/lucia/miniconda3/envs/3.10/bin:/mnt/ssd-1/lucia/miniconda3/condabin:/mnt/ssd-1/lucia/miniconda3/bin:/home/lucia/.cargo/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin', 'VSCODE_AGENT_FOLDER': '/home/lucia/.vscode-server', 'SSL_CERT_DIR': '/usr/lib/ssl/certs', 'SSH_AUTH_SOCK': '/tmp/ssh-M4NXPRM9Yq/agent.1861034', 'SHELL': '/bin/zsh', 'PWD': '/home/lucia', 'SSH_CONNECTION': '119.18.1.138 12430 10.147.243.154 22', 'VSCODE

In [2]:
dataset = load_dataset("allenai/c4", "es", split='train', download_config=DownloadConfig(resume_download=True)) # columns=['text']

In [13]:
# es = load_dataset("allenai/c4", "es", split='train')
es = load_dataset("es/pile-10k", split='train')

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

data, bpb_ratio = chunk_and_tokenize(es, tokenizer, max_length=2049)
data.save_to_disk("es_tokenized.hf")


print(data["length"][0:6])
print(len(data["input_ids"][0]), len(data["input_ids"][1]), len(data["input_ids"][2]), len(data["input_ids"][4]), len(data["input_ids"][6]))
print(sum(data["length"]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Saving the dataset (1/1 shards): 100%|██████████| 7492/7492 [00:00<00:00, 44910.60 examples/s]


[2050, 2050, 2050, 2050, 2050, 2050]
2049 2049 2049 2049 2049
15360451


In [19]:
print(os.getcwd())

/home/lucia/features-across-time/scripts


In [17]:
# first = 0
# sec = 0
# third = 0
# four = 0
# for item in data['length']:
#     if item == 2048:
#         first += 1
#     elif item == 2049:
#         sec += 1
#     elif item == 2050:
#         third += 1
#     elif item == 2051:
#         four += 1
#     else:
#         print(item)

# print(first, sec, third, four)
print(len(data))

7492


In [18]:
fp = np.memmap('es.bin', dtype=np.uint16, mode='w+', shape=(len(data), 2049))
for i, item in enumerate(data):
    fp[i] = item['input_ids'].numpy()
fp.flush()